Testing Jun2 

Base test for BRK.B 

In [36]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np 
from IPython.display import display


In [37]:
#Dataset 
file_path = '/Users/schoudhry/Desktop/shashy/traderAuto/BRK.B(jan9.2024).csv'
df = pd.read_csv(file_path)

# Clean up column names and format date
df.columns = df.columns.str.strip()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Display the entire DataFrame (remove row limit)
pd.set_option('display.max_rows', None)
display(df)

,Date,Open,High,Low,Close,Volume
348,2024-01-09,368.000,368.0000,364.7700,366.90,2999607
347,2024-01-10,366.260,368.8300,365.7500,367.92,2448737
346,2024-01-11,367.810,367.8900,362.1300,363.34,3579713
345,2024-01-12,366.510,366.8500,362.5672,363.62,2623907
344,2024-01-16,362.940,363.6000,359.8900,361.10,3047780
343,2024-01-17,359.010,362.5300,357.9800,359.29,2551707
342,2024-01-18,359.250,362.6400,358.3000,362.38,3008442
341,2024-01-19,362.250,368.3100,361.0200,366.99,4092548
340,2024-01-22,368.050,369.2700,366.5300,368.06,3558313
339,2024-01-23,368.020,372.4200,367.7050,372.14,2900467


In [38]:
#Candlestick for BRK.B 

fig = go.Figure(data=[go.Candlestick(
    x=df['Date'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='BRK.B'
)])

fig.update_layout(
    title='BRK.B Daily Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    xaxis_rangeslider_visible=False,
    template='plotly_white',
    height=600
)

fig.show()
    

In [39]:
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

# === Constants ===
initial_price = 468.5
initial_investment = 1000.0
buy_amount = 125.0
sell_amount = 100.0
latest_close = df.iloc[-1]['Close']

# === Strategy Simulation Function ===
def simulate_strategy(df, buy_thresh, sell_thresh):
    cash = 0.0
    shares = initial_investment / initial_price
    trades = 0

    for _, row in df.iterrows():
        low = row['Low']
        high = row['High']

        # Buy logic
        if low <= buy_thresh:
            shares_bought = buy_amount / buy_thresh
            shares += shares_bought
            cash -= buy_amount
            trades += 1

        # Sell logic
        if high >= sell_thresh:
            shares_to_sell = sell_amount / sell_thresh
            if shares >= shares_to_sell:
                shares -= shares_to_sell
                cash += sell_amount
                trades += 1

    final_value = cash + shares * latest_close
    net_gain = final_value - initial_investment
    return final_value, net_gain, trades

# === IRL Strategy ===
irl_value, irl_gain, irl_trades = simulate_strategy(df, 495.5, 505.95)

# === AI Strategy Optimization ===
best_value = 0
best_buy = None
best_sell = None
best_trades = 0

for buy_thresh in np.arange(480, 500.5, 0.5):
    for sell_thresh in np.arange(500, 520.5, 0.5):
        value, gain, trades = simulate_strategy(df, buy_thresh, sell_thresh)
        if value > best_value:
            best_value = value
            best_buy = buy_thresh
            best_sell = sell_thresh
            best_trades = trades

# === Result Comparison Table ===
comparison = pd.DataFrame([
    {
        "Strategy": "IRL Strategy",
        "Buy Threshold": 495.5,
        "Sell Threshold": 505.95,
        "Final Value": round(irl_value, 2),
        "Net Gain": round(irl_gain, 2),
        "Return (%)": round((irl_gain / initial_investment) * 100, 2),
        "Total Trades": irl_trades
    },
    {
        "Strategy": "AI-Optimized Strategy",
        "Buy Threshold": best_buy,
        "Sell Threshold": best_sell,
        "Final Value": round(best_value, 2),
        "Net Gain": round(best_value - initial_investment, 2),
        "Return (%)": round(((best_value - initial_investment) / initial_investment) * 100, 2),
        "Total Trades": best_trades
    }
])

display(comparison)




,Strategy,Buy Threshold,Sell Threshold,Final Value,Net Gain,Return (%),Total Trades
0,IRL Strategy,495.5,505.95,1729.44,729.44,72.94,352
1,AI-Optimized Strategy,480.0,520.00,2924.49,1924.49,192.45,314


In [40]:
# Generate AI and IRL detailed logs (include shares, cash, portfolio value)
def get_detailed_trade_log(df, buy_thresh, sell_thresh, label):
    cash = 0.0
    shares = initial_investment / initial_price
    trade_log = []

    for _, row in df.iterrows():
        date = row['Date']
        low = row['Low']
        high = row['High']
        close = row['Close']

        if low <= buy_thresh:
            shares_bought = buy_amount / buy_thresh
            shares += shares_bought
            cash -= buy_amount
            trade_log.append({
                "Date": date,
                "Strategy": label,
                "Action": "Buy",
                "Price": buy_thresh,
                "Shares Traded": shares_bought,
                "Cash Change": -buy_amount,
                "Shares Held": shares,
                "Cash": cash,
                "Portfolio Value": cash + shares * close
            })

        if high >= sell_thresh:
            shares_to_sell = sell_amount / sell_thresh
            if shares >= shares_to_sell:
                shares -= shares_to_sell
                cash += sell_amount
                trade_log.append({
                    "Date": date,
                    "Strategy": label,
                    "Action": "Sell",
                    "Price": sell_thresh,
                    "Shares Traded": -shares_to_sell,
                    "Cash Change": sell_amount,
                    "Shares Held": shares,
                    "Cash": cash,
                    "Portfolio Value": cash + shares * close
                })

    return pd.DataFrame(trade_log)

# Create detailed logs to enable the merge
ai_log_detailed = get_detailed_trade_log(df, best_buy, best_sell, "AI Strategy")
irl_log_detailed = get_detailed_trade_log(df, 495.5, 505.95, "IRL Strategy")

ai_merge = ai_log_detailed[["Date", "Action", "Price"]].copy()
ai_merge.columns = ["Date", "AI Action", "AI Price"]

irl_merge = irl_log_detailed[["Date", "Action", "Price"]].copy()
irl_merge.columns = ["Date", "IRL Action", "IRL Price"]

comparison_table = pd.merge(ai_merge, irl_merge, on="Date", how="outer")
comparison_table = comparison_table.sort_values("Date").reset_index(drop=True)

display(comparison_table)



,Date,AI Action,AI Price,IRL Action,IRL Price
0,2024-01-09,Buy,480.0,Buy,495.50
1,2024-01-10,Buy,480.0,Buy,495.50
2,2024-01-11,Buy,480.0,Buy,495.50
3,2024-01-12,Buy,480.0,Buy,495.50
4,2024-01-16,Buy,480.0,Buy,495.50
5,2024-01-17,Buy,480.0,Buy,495.50
6,2024-01-18,Buy,480.0,Buy,495.50
7,2024-01-19,Buy,480.0,Buy,495.50
8,2024-01-22,Buy,480.0,Buy,495.50
9,2024-01-23,Buy,480.0,Buy,495.50
